1. Rainbow and Ape-X Expiriments 
    1. We release a set of hyper parameters for CartPole-v1 and Classic Control and Atari
    2. We release code for Rainbow that can train X steps in Y minutes on a Mac M2 Chip
    3. We also release a version of Ape-X as described in the original paper, and an Ape-X with rainbow
        1. Compare results of each 
        2. Compare Ape-X with different Rainbow components added or removed
    4. We compare the different models of DQN as seen in their papers to rainbow, the different individual components to rainbow, and rainbow with individual components removed
    10. Compare rainbow training speeds with different levels of numerical precision and datatypes
        1. Mixed precision using torch.amp 
        2. Lower matmul precision
            1. comparing medium, high, and highest 
            2. https://pytorch.org/docs/master/generated/torch.set_float32_matmul_precision.html?highlight=precision#torch.set_float32_matmul_precision
    11. Ape-X Hyper parameter sweep and sensitivities
    12. Exploration methods for Rainbow Ape-X
        1. Just noisy nets (same for all actors)
        2. Noisy nets and varying epsilon 
        3. Adding a constant that changes variance of noisy nets for action selection
        4. AlphaStar Agents

## Rainbow on CartPole-v1
Hyperparameters are based on the hyperopt experiments, quantized trial 27 with some minor changes

In [1]:
import gymnasium as gym
import sys

import torch
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import CartPoleConfig

env = gym.make("CartPole-v1", render_mode="rgb_array")

config_dict = {
  "dense_layers_widths": [128, 128],
  "value_hidden_layers_widths": [64, 64],
  "advatage_hidden_layers_widths": [64, 64],
  "adam_epsilon": 0.00375,
  "learning_rate": 0.005,
  "training_steps": 10000,
  "per_epsilon": 0.05,
  "per_alpha": 0.8,
  "per_beta": 0.45,
  "minibatch_size": 128,
  "replay_buffer_size": 10000,
  "min_replay_buffer_size": 1250,
  "transfer_interval": 10,
  "n_step": 9,
  "kernel_initializer": "glorot_uniform",
  "loss_function": KLDivergenceLoss(), # could do categorical cross entropy 
  "clipnorm": 2.0,
  "discount_factor": 0.99,
  "atom_size": 81,
  "replay_interval": 4,
}
game_config = CartPoleConfig()
config = RainbowConfig(config_dict, game_config)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
agent = RainbowAgent(env, config, name="Rainbow_CartPole-v1", device=device)

for param in agent.model.parameters():
  print(param)
print("start")
agent.train()

ImportError: cannot import name 'CategoricalCrossentropy' from 'utils' (/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/__init__.py)

## Rainbow on Classic Control
Hyperparameters come from revisitting rainbow

In [4]:
import gymnasium as gym
import sys
import torch 

from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import ClassicControlConfig

config_dict = {
  "dense_layers_widths": [512, 512],
  "value_hidden_layers_widths": [], # 
  "advatage_hidden_layers_widths": [], # 
  "adam_epsilon": 3.125e-4,
  "learning_rate": 0.001,
  "training_steps": 30000,
  "per_epsilon": 1e-6, # 
  "per_alpha": 0.5,
  "per_beta": 0.5, # For RIAYN should be no annealing # 0.4
  "minibatch_size": 128,
  "replay_buffer_size": 50000,
  "min_replay_buffer_size": 500,
  "transfer_interval": 100,
  "n_step": 3,
  "kernel_initializer": "orthogonal", #
  "loss_function": KLDivergenceLoss(), # KLDivergence()
  "clipnorm": 0.0, # 2.0 
  "discount_factor": 0.99,
  "atom_size": 51,
  "replay_interval": 2,
}
game_config = ClassicControlConfig()
config = RainbowConfig(config_dict, game_config)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# game_config.v_min = 0
# game_config.v_max = 500
# env = gym.make("CartPole-v1", render_mode="rgb_array")
# agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_CartPole-v1", device=device)
# agent.train()

# game_config.v_min = -500
# game_config.v_max = 0
# env = gym.make("Acrobot-v1", render_mode="rgb_array")
# agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_Acrobot-v1", device=device)
# agent.train()

# game_config.v_min = -200
# game_config.v_max = 200
# env = gym.make("LunarLander-v2", render_mode="rgb_array")
# agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_LunarLander-v2", device=device)
# agent.train()

# game_config.v_min = -200
# game_config.v_max = -100
env = gym.make("MountainCar-v0", render_mode="rgb_array", max_episode_steps=600)
agent = RainbowAgent(env, config, name="Rainbow_ClassicControl_MountainCar-v0", device=device)
agent.train()

Using default save_intermediate_weights     : False
Using         training_steps                : 30000
Using         adam_epsilon                  : 0.0003125
Using         learning_rate                 : 0.001
Using         clipnorm                      : 0.0
Using default optimizer                     : <class 'torch.optim.adam.Adam'>
Using         loss_function                 : <utils.utils.KLDivergenceLoss object at 0x103812ff0>
Using default activation                    : relu
Using         kernel_initializer            : orthogonal
Using         minibatch_size                : 128
Using         replay_buffer_size            : 50000
Using         min_replay_buffer_size        : 500
Using default num_minibatches               : 1
Using default training_iterations           : 1
Using default conv_layers                   : []
Using         dense_layers_widths           : [512, 512]
Using         value_hidden_layers_widths    : []
Using default advantage_hidden_layers_widths: []
U

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/rainbow_and_ape-x folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


score:  -600.0
score:  -600.0
score:  -600.0
score:  -600.0
Moviepy - Building video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_1000/videos/Rainbow_ClassicControl_MountainCar-v0/1000/Rainbow_ClassicControl_MountainCar-v0-episode-4.mp4.
Moviepy - Writing video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_1000/videos/Rainbow_ClassicControl_MountainCar-v0/1000/Rainbow_ClassicControl_MountainCar-v0-episode-4.mp4



Moviepy - Done !
Moviepy - video ready checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_1000/videos/Rainbow_ClassicControl_MountainCar-v0/1000/Rainbow_ClassicControl_MountainCar-v0-episode-4.mp4
score:  -600.0


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:255: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()
/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:185: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  axs[row][col].set_xlim(1, len(values))


...

/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:255: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


score:  -87.0
score:  -600.0
score:  -85.0
score:  -92.0
Moviepy - Building video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29999/videos/Rainbow_ClassicControl_MountainCar-v0/29999/Rainbow_ClassicControl_MountainCar-v0-episode-149.mp4.
Moviepy - Writing video checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29999/videos/Rainbow_ClassicControl_MountainCar-v0/29999/Rainbow_ClassicControl_MountainCar-v0-episode-149.mp4



Moviepy - Done !
Moviepy - video ready checkpoints/Rainbow_ClassicControl_MountainCar-v0/step_29999/videos/Rainbow_ClassicControl_MountainCar-v0/29999/Rainbow_ClassicControl_MountainCar-v0-episode-149.mp4
score:  -129.0


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:255: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


: 

## Rainbow on Atari
Takes around 18GB RAM, and many hours even just to fill replay buffer with a Mac M2

### Rainbow MsPacman

In [ ]:
import gymnasium as gym
import sys

from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import AtariConfig
from gymnasium.wrappers import AtariPreprocessing, FrameStack
import numpy as np

config_dict = {
  "conv_layers": [
      (32, 8, 4),
      (64, 4, 2),
      (64, 3, 1),
  ],
  "dense_layers_widths": [512], 
  "value_hidden_layers_widths": [], # 
  "advatage_hidden_layers_widths": [], # 
  "adam_epsilon": 1.5e-4,
  "learning_rate": 0.00025/4,
  "training_steps": 50000000, # Agent saw 200,000,000 frames
  "per_epsilon": 1e-6, # 
  "per_alpha": 0.5, 
  "per_beta": 0.4, 
  "minibatch_size": 32,
  "replay_buffer_size": 1000000,
  "min_replay_buffer_size": 80000,
  "transfer_interval": 32000,
  "n_step": 3,
  "kernel_initializer": "orthogonal", #
  "loss_function": KLDivergenceLoss(),
  "clipnorm": 0.0, # 
  "discount_factor": 0.99,
  "atom_size": 51,
  "replay_interval": 4,
}
game_config = AtariConfig()
config = RainbowConfig(config_dict, game_config)

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)
env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array", max_episode_steps=108000)
env = AtariPreprocessing(env, terminal_on_life_loss=True)
env = FrameStack(env, 4)
agent = RainbowAgent(env, config, name="Rainbow_Atari_MsPacmanNoFrameskip-v4")
agent.checkpoint_interval = 1000
agent.train()

## Rainbow with  turn-based zero-sum 2-player perfect information deterministic games
Not working yet because of zero sum rewards, could get it to work by doing an NFSP sort of thing when storing experiences

### Rainbow Tic Tac Toe

#### Framestacking

In [ ]:
import gymnasium as gym
import sys

from packages.game_configs.game_configs.tictactoe_config import TicTacToeConfig
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import AtariConfig
from gymnasium.wrappers import AtariPreprocessing, FrameStack
import numpy as np

config_dict = {
  "conv_layers": [
      (32, 8, 4),
      (64, 4, 2),
      (64, 3, 1),
  ],
  "dense_layers_widths": [512], 
  "value_hidden_layers_widths": [], # 
  "advatage_hidden_layers_widths": [], # 
  "adam_epsilon": 1.5e-4,
  "learning_rate": 0.00025/4,
  "training_steps": 50000000, # Agent saw 200,000,000 frames
  "per_epsilon": 1e-6, # 
  "per_alpha": 0.5, 
  "per_beta": 0.4, 
  "minibatch_size": 32,
  "replay_buffer_size": 1000000,
  "min_replay_buffer_size": 80000,
  "transfer_interval": 32000,
  "n_step": 3,
  "kernel_initializer": "orthogonal", #
  "loss_function": KLDivergenceLoss(),
  "clipnorm": 0.0, # 
  "discount_factor": 0.99,
  "atom_size": 51,
  "replay_interval": 4,
}
game_config = TicTacToeConfig()
config = RainbowConfig(config_dict, game_config)

env = gym.make("TicTacToe-v0", render_mode="rgb_array")
env = FrameStack(env, 4)
agent = RainbowAgent(env, config, name="Rainbow_TicTacToe-v0")
agent.train()

#### No framestacking

In [ ]:
import gymnasium as gym
import sys

from packages.game_configs.game_configs.tictactoe_config import TicTacToeConfig
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
sys.path.append('../..')
from dqn.rainbow.rainbow_agent import RainbowAgent
from agent_configs import RainbowConfig
from game_configs import AtariConfig
from gymnasium.wrappers import AtariPreprocessing, FrameStack
import numpy as np

config_dict = {
  "conv_layers": [
      (32, 8, 4),
      (64, 4, 2),
      (64, 3, 1),
  ],
  "dense_layers_widths": [512], 
  "value_hidden_layers_widths": [], # 
  "advatage_hidden_layers_widths": [], # 
  "adam_epsilon": 1.5e-4,
  "learning_rate": 0.00025/4,
  "training_steps": 50000000, # Agent saw 200,000,000 frames
  "per_epsilon": 1e-6, # 
  "per_alpha": 0.5, 
  "per_beta": 0.4, 
  "minibatch_size": 32,
  "replay_buffer_size": 1000000,
  "min_replay_buffer_size": 80000,
  "transfer_interval": 32000,
  "n_step": 3,
  "kernel_initializer": "orthogonal", #
  "loss_function": KLDivergenceLoss(),
  "clipnorm": 0.0, # 
  "discount_factor": 0.99,
  "atom_size": 51,
  "replay_interval": 4,
}
game_config = TicTacToeConfig()
config = RainbowConfig(config_dict, game_config)

env = gym.make("TicTacToe-v0", render_mode="rgb_array")
agent = RainbowAgent(env, config, name="Rainbow_TicTacToe-v0")
agent.train()

### Rainbow Connect 4

### Rainbow Chess